# Annihilation operator for a single boson

The cutoff L needs to be a power of 2 because we want to use qubit operators.

In [1]:
import numpy as np
from scipy.sparse import diags
L = 4  # cutoff for Fock space
a = diags(np.sqrt(np.linspace(1,L-1,L-1)),offsets=1)

In [2]:
print(a.toarray())

[[0.         1.         0.         0.        ]
 [0.         0.         1.41421356 0.        ]
 [0.         0.         0.         1.73205081]
 [0.         0.         0.         0.        ]]


## Representation in QISKIT

In [3]:
from qiskit.opflow import MatrixOp

qubitOp = MatrixOp(primitive=a)
print(qubitOp.num_qubits)

2


In [5]:
type(qubitOp)

qiskit.opflow.primitive_ops.matrix_op.MatrixOp

In [6]:
print(qubitOp)

Operator([[0.        +0.j, 1.        +0.j, 0.        +0.j, 0.        +0.j],
          [0.        +0.j, 0.        +0.j, 1.41421356+0.j, 0.        +0.j],
          [0.        +0.j, 0.        +0.j, 0.        +0.j, 1.73205081+0.j],
          [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j]],
         input_dims=(2, 2), output_dims=(2, 2))


Express the annihilation operator of a single boson in terms of N-qubit operators represented by Pauli matrices (2x2)

In [7]:
a_pauli = qubitOp.to_pauli_op()
print(a_pauli)

SummedOp([
  0.6830127018922193 * IX,
  0.6830127018922193j * IY,
  0.3535533905932738 * XX,
  -0.3535533905932738j * XY,
  0.3535533905932738j * YX,
  0.3535533905932738 * YY,
  -0.1830127018922193 * ZX,
  -0.1830127018922193j * ZY
])


In [8]:
type(a_pauli)

qiskit.opflow.list_ops.summed_op.SummedOp

In [9]:
a_pauli.to_matrix()

array([[0.        +0.j, 1.        +0.j, 0.        +0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 1.41421356+0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.        +0.j, 1.73205081+0.j],
       [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j]])

# Identity operator for a single boson

In [10]:
from scipy.sparse import identity
iden = identity(L)

In [11]:
qubitOp = MatrixOp(primitive=iden)
print(qubitOp.num_qubits)

2


In [12]:
i_pauli = qubitOp.to_pauli_op()
print(i_pauli)

II


In [13]:
i_pauli.to_matrix()

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

# Create the Hamiltonian for 6 interacting bosons

Try to use the information in [this QISKIT tutorial](https://qiskit.org/documentation/tutorials/operators/01_operator_flow.html)

In [14]:
Nmat = 6  # BMN2 for SU(2) has 6 bosonic matrices

In [15]:
from qiskit.opflow import ListOp, TensoredOp

boson = ListOp([i_pauli]*6)
print(boson)

ListOp([
  II,
  II,
  II,
  II,
  II,
  II
])


In [16]:
print(boson[0])

II


We can use a list of `ListOp` and then perform the tensor product using `TensoredOp`

In [17]:
# generically speaking, we construct the list of bosons and then take the outer product
a_list = []  # this will contain a1...a6 as a list of ListOp
for i in np.arange(0,Nmat):  # loop over all operators
    operator_list = [i_pauli] * Nmat  # only the identity repeated Nmat times
    operator_list[i] = a_pauli  # the i^th element is now the annihilation operator for a single boson
    a_list.append(ListOp(operator_list))


In [18]:
print(a_list[0])

ListOp([
  SummedOp([
    0.6830127018922193 * IX,
    0.6830127018922193j * IY,
    0.3535533905932738 * XX,
    -0.3535533905932738j * XY,
    0.3535533905932738j * YX,
    0.3535533905932738 * YY,
    -0.1830127018922193 * ZX,
    -0.1830127018922193j * ZY
  ]),
  II,
  II,
  II,
  II,
  II
])


In [19]:
# here we create a list of operators which are the tensor products...
a_tensor = [TensoredOp(x) for x in a_list]


In [20]:
print(a_tensor[0])

TensoredOp([
  SummedOp([
    0.6830127018922193 * IX,
    0.6830127018922193j * IY,
    0.3535533905932738 * XX,
    -0.3535533905932738j * XY,
    0.3535533905932738j * YX,
    0.3535533905932738 * YY,
    -0.1830127018922193 * ZX,
    -0.1830127018922193j * ZY
  ]),
  II,
  II,
  II,
  II,
  II
])


In [21]:
print(a_tensor[0].to_pauli_op())

0.6830127018922193 * IXIIIIIIIIII
- 0.1830127018922193 * ZXIIIIIIIIII
+ 0.6830127018922193j * IYIIIIIIIIII
+ -0.1830127018922193j * ZYIIIIIIIIII
+ 0.3535533905932738 * XXIIIIIIIIII
+ 0.3535533905932738j * YXIIIIIIIIII
+ -0.3535533905932738j * XYIIIIIIIIII
+ 0.3535533905932738 * YYIIIIIIIIII


The representation is already in terms of Pauli operators.

In [22]:
a_tensor[0].num_qubits

12

In [23]:
sparse = a_tensor[0].to_spmatrix()
type(sparse)

scipy.sparse.csr.csr_matrix

In [24]:
sparse.shape

(4, 4)

In [25]:
full = a_tensor[0].to_matrix()
len(full)

4096

In [26]:
full.shape

(4096, 4096)

We also create the identity in this new tensor space of 6 bosons

In [27]:
i_tensor = TensoredOp(ListOp([i_pauli] * Nmat))

In [41]:
type(i_tensor)

qiskit.opflow.list_ops.tensored_op.TensoredOp

In [28]:
print(i_tensor)

TensoredOp([
  II,
  II,
  II,
  II,
  II,
  II
])


In [29]:
i_tensor.num_qubits

12

In [30]:
full = i_tensor.to_matrix()
len(full)

4096

In [31]:
print(i_tensor.to_spmatrix())

  (0, 0)	(1+0j)
  (1, 1)	(1+0j)
  (2, 2)	(1+0j)
  (3, 3)	(1+0j)


## Create the position operators

In [32]:
# for each boson they are constructed using a and adag
x_tensor = [1/np.sqrt(2)*(~x + x) for x in a_tensor]

In [33]:
print(x_tensor[0])

0.7071067811865475 * SummedOp([
  TensoredOp([
    SummedOp([
      0.6830127018922193 * IX,
      -0.6830127018922193j * IY,
      0.3535533905932738 * XX,
      0.3535533905932738j * XY,
      -0.3535533905932738j * YX,
      0.3535533905932738 * YY,
      -0.1830127018922193 * ZX,
      0.1830127018922193j * ZY
    ]),
    II,
    II,
    II,
    II,
    II
  ]),
  TensoredOp([
    SummedOp([
      0.6830127018922193 * IX,
      0.6830127018922193j * IY,
      0.3535533905932738 * XX,
      -0.3535533905932738j * XY,
      0.3535533905932738j * YX,
      0.3535533905932738 * YY,
      -0.1830127018922193 * ZX,
      -0.1830127018922193j * ZY
    ]),
    II,
    II,
    II,
    II,
    II
  ])
])


In [34]:
print(x_tensor[0].to_pauli_op())

SummedOp([
  0.9659258262890682 * IXIIIIIIIIII,
  -0.2588190451025207 * ZXIIIIIIIIII,
  0.5 * XXIIIIIIIIII,
  0.5 * YYIIIIIIIIII
])


In [35]:
print(x_tensor[0].to_spmatrix())

  (0, 1)	(0.7071067811865474+0j)
  (1, 0)	(0.7071067811865474+0j)
  (1, 2)	(1+0j)
  (2, 1)	(1+0j)
  (2, 3)	(1.224744871391589+0j)
  (3, 2)	(1.224744871391589+0j)


## Create the full Hamiltonian

In [36]:
from qiskit.opflow import SummedOp

In [37]:
H_zero = 0.5*Nmat*i_tensor

In [38]:
print(H_zero)

3.0 * TensoredOp([
  II,
  II,
  II,
  II,
  II,
  II
])


In [39]:
H_zero.to_spmatrix()

<4x4 sparse matrix of type '<class 'numpy.complex128'>'
	with 4 stored elements in Compressed Sparse Row format>

In [40]:
H_zero.to_pauli_op()

PauliOp(Pauli('IIIIIIIIIIII'), coeff=3.0)

In [42]:
### Harmonic oscillator
# this should be summed over all the bosons (Nmat)
H_list = [H_zero]
for op in a_tensor:
    H_list.append((~op @ op))
H_osc = SummedOp(H_list)


In [43]:
print(H_osc)

SummedOp([
  3.0 * TensoredOp([
    II,
    II,
    II,
    II,
    II,
    II
  ]),
  ComposedOp([
    TensoredOp([
      SummedOp([
        0.6830127018922193 * IX,
        -0.6830127018922193j * IY,
        0.3535533905932738 * XX,
        0.3535533905932738j * XY,
        -0.3535533905932738j * YX,
        0.3535533905932738 * YY,
        -0.1830127018922193 * ZX,
        0.1830127018922193j * ZY
      ]),
      II,
      II,
      II,
      II,
      II
    ]),
    TensoredOp([
      SummedOp([
        0.6830127018922193 * IX,
        0.6830127018922193j * IY,
        0.3535533905932738 * XX,
        -0.3535533905932738j * XY,
        0.3535533905932738j * YX,
        0.3535533905932738 * YY,
        -0.1830127018922193 * ZX,
        -0.1830127018922193j * ZY
      ]),
      II,
      II,
      II,
      II,
      II
    ])
  ]),
  ComposedOp([
    TensoredOp([
      II,
      SummedOp([
        0.6830127018922193 * IX,
        -0.6830127018922193j * IY,
        0.3535533905932738

In [44]:
type(H_osc)

qiskit.opflow.list_ops.summed_op.SummedOp

In [59]:
H_osc_pauli = H_osc.to_pauli_op()

In [60]:
type(H_osc_pauli)

qiskit.opflow.list_ops.summed_op.SummedOp

In [61]:
H_osc_pauli.num_qubits

12

In [48]:
H_osc.to_spmatrix()

<4x4 sparse matrix of type '<class 'numpy.complex128'>'
	with 4 stored elements in Compressed Sparse Row format>

In [49]:
quartic1 = SummedOp([x_tensor[2]@x_tensor[2]@x_tensor[3]@x_tensor[3], \
                    x_tensor[2]@x_tensor[2]@x_tensor[4]@x_tensor[4], \
                    x_tensor[1]@x_tensor[1]@x_tensor[3]@x_tensor[3], \
                    x_tensor[1]@x_tensor[1]@x_tensor[5]@x_tensor[5], \
                    x_tensor[0]@x_tensor[0]@x_tensor[4]@x_tensor[4], \
                    x_tensor[0]@x_tensor[0]@x_tensor[5]@x_tensor[5]])
                    

In [50]:
print(quartic1.to_pauli_op())

SummedOp([
  13.499999999999998 * IIIIIIIIIIII,
  -1.4999999999999993 * ZIIIIIIIIIII,
  -1.4999999999999998 * ZZIIIIIIIIII,
  -1.4999999999999993 * IIZIIIIIIIII,
  -1.4999999999999998 * IIZZIIIIIIII,
  -1.4999999999999993 * IIIIZIIIIIII,
  -1.4999999999999998 * IIIIZZIIIIII,
  -1.4999999999999993 * IIIIIIZIIIII,
  0.24999999999999978 * IIZIIIZIIIII,
  0.2499999999999999 * IIZZIIZIIIII,
  0.24999999999999978 * IIIIZIZIIIII,
  0.2499999999999999 * IIIIZZZIIIII,
  -1.4999999999999998 * IIIIIIZZIIII,
  0.2499999999999999 * IIZIIIZZIIII,
  0.25 * IIZZIIZZIIII,
  0.2499999999999999 * IIIIZIZZIIII,
  0.25 * IIIIZZZZIIII,
  -1.4999999999999993 * IIIIIIIIZIII,
  0.24999999999999978 * ZIIIIIIIZIII,
  0.2499999999999999 * ZZIIIIIIZIII,
  0.24999999999999978 * IIIIZIIIZIII,
  0.2499999999999999 * IIIIZZIIZIII,
  -1.4999999999999998 * IIIIIIIIZZII,
  0.2499999999999999 * ZIIIIIIIZZII,
  0.25 * ZZIIIIIIZZII,
  0.2499999999999999 * IIIIZIIIZZII,
  0.25 * IIIIZZIIZZII,
  -1.4999999999999993 * IIIIIIII

In [51]:
quartic2 = SummedOp([x_tensor[0]@x_tensor[2]@x_tensor[3]@x_tensor[5],\
                    x_tensor[0]@x_tensor[1]@x_tensor[3]@x_tensor[4],\
                    x_tensor[1]@x_tensor[2]@x_tensor[4]@x_tensor[5]], coeff = -2.0)

In [52]:
print(quartic2.to_pauli_op())

SummedOp([
  -1.7410254037844382 * IXIXIIIXIXII,
  0.46650635094610937 * ZXIXIIIXIXII,
  0.46650635094610937 * IXZXIIIXIXII,
  -0.1249999999999999 * ZXZXIIIXIXII,
  0.4665063509461094 * IXIXIIZXIXII,
  -0.1249999999999999 * ZXIXIIZXIXII,
  -0.1249999999999999 * IXZXIIZXIXII,
  0.0334936490538903 * ZXZXIIZXIXII,
  0.4665063509461094 * IXIXIIIXZXII,
  -0.12499999999999989 * ZXIXIIIXZXII,
  -0.12499999999999989 * IXZXIIIXZXII,
  0.0334936490538903 * ZXZXIIIXZXII,
  -0.1249999999999999 * IXIXIIZXZXII,
  0.0334936490538903 * ZXIXIIZXZXII,
  0.0334936490538903 * IXZXIIZXZXII,
  -0.008974596215561343 * ZXZXIIZXZXII,
  -0.9012210650134379 * XXIXIIIXIXII,
  -0.9012210650134379 * YYIXIIIXIXII,
  0.24148145657226694 * XXZXIIIXIXII,
  0.24148145657226694 * YYZXIIIXIXII,
  0.24148145657226694 * XXIXIIZXIXII,
  0.24148145657226694 * YYIXIIZXIXII,
  -0.06470476127563014 * XXZXIIZXIXII,
  -0.06470476127563014 * YYZXIIZXIXII,
  0.24148145657226697 * XXIXIIIXZXII,
  0.24148145657226697 * YYIXIIIXZXII,
 

In [53]:
type(quartic2)

qiskit.opflow.list_ops.summed_op.SummedOp

In [54]:
### Quartic Interaction
V =  quartic1 + quartic2

In [55]:
V.num_qubits

12

In [57]:
V_pauli = V.to_pauli_op()

## Define the 'tHooft coupling

In [62]:
g2N = 0.2
N = 2

In [63]:
### Full Hamiltonian
H = H_osc + g2N / N * V

In [64]:
type(H)

qiskit.opflow.list_ops.summed_op.SummedOp

In [ ]:
H.to_pauli_op()

In [66]:
type(H.to_pauli_op())

qiskit.opflow.list_ops.summed_op.SummedOp

In [67]:
H.num_qubits

12

In [68]:
testH = H.to_matrix_op()
testH

KeyboardInterrupt: 

If we do not do this conversion for each term, we do not get the correct result!

In [69]:
Hpauli = H.to_pauli_op()

# the line below is also fine...
# Hpauli1 = H_osc.to_pauli_op() + g2N / N * V.to_pauli_op()
# Hpauli

In [79]:
type(Hpauli)

qiskit.opflow.list_ops.summed_op.SummedOp

In [82]:
# write expression to disk
with open('pauliH.txt','w') as f:
    print(Hpauli,file=f)

# Diagonalize the Hamiltonian to find the groundstate

In [72]:
from qiskit.algorithms import NumPyMinimumEigensolver

npme = NumPyMinimumEigensolver()

In [76]:
# result = npme.compute_minimum_eigenvalue(operator=H_osc)
# result = npme.compute_minimum_eigenvalue(operator=H)
# result = npme.compute_minimum_eigenvalue(operator=Hpauli)
result = npme.compute_minimum_eigenvalue(operator=Hpauli1)
# result = npme.compute_minimum_eigenvalue(operator=testH)

In [77]:
print(result)

{   'aux_operator_eigenvalues': None,
    'eigenstate': VectorStateFn(Statevector([ 8.97501246e-01+4.34640370e-01j,
              4.54838201e-17+5.56838331e-16j,
             -2.58033301e-02-1.24959926e-02j, ...,
              1.89036666e-18+1.58108432e-18j,
              2.55012674e-17-2.44419602e-18j,
             -6.66962171e-18-2.56864339e-18j],
            dims=(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2)), coeff=1.0, is_measurement=False),
    'eigenvalue': (3.134063071928589+2.2499258724465427e-16j)}


In [78]:
ref_value = result.eigenvalue.real
print(f'Ground state energy value: {ref_value:.5f}')

Ground state energy value: 3.13406


I can not get anything different from 3.000 even if I have the interactions turned on... The reason I found out is that the `MatrixOp` is interpreted differently than a `SummedOp` by the eigensolver. When using the `SummedOp` that I get from the `PauliOp` procedure, it turns out that the statevector is just living in a 2-dimensional space (1 qubit) instead of the full tensor product space of 6 qubits:
``` 
dims=(2,)) instead of dims=(2, 2, 2, 2, 2, 2))
```

One solution is to create the Hamiltonian calling `.to_pauli_op()` on each term.

## Construct the Hamiltonian matrix as usual
This only works for cutoffs smaller than 5 due to memory requirements in `MatrixOp`.

In [21]:
from scipy.sparse import kron 

def build_operators(L: int, N: int) -> list:
    """Generate all the annihilation operators needed to build the hamiltonian
    Args:
        L (int): the cutoff of the single site Fock space
        N (int): the number of colors of gauge group SU(N)
    Returns:
        list: a list of annihilation operators, length=N_bos
    """
    # The annihilation operator for the single boson
    a_b = diags(np.sqrt(np.linspace(1, L - 1, L - 1)), offsets=1)
    # The identity operator of the Fock space of a single boson
    i_b = identity(L)
    # Bosonic Hilbert space
    N_bos = int(2 * (N ** 2 - 1))  # number of boson sites -> fixed for mini-BMN 2
    product_list = [i_b] * N_bos  # only the identity for bosons repeated N_bos times
    a_b_list = []  # this will contain a1...a6
    for i in np.arange(0, N_bos):  # loop over all bosonic operators
        operator_list = product_list.copy()  # all elements are the identity operator
        operator_list[
            i
        ] = a_b  # the i^th element is now the annihilation operator for a single boson
        a_b_list.append(
            operator_list[0]
        )  # start taking tensor products from first element
        for a in operator_list[1:]:
            a_b_list[i] = kron(
                a_b_list[i], a
            )  # do the outer product between each operator_list element
    return a_b_list

In [35]:
N = 2  # group SU(2)
a_b_list = build_operators(L,N)
# Build the Hamiltonian
# Start piece by piece
x_list = []
# only use the bosonic operators
for op in a_b_list:
    x_list.append(1 / np.sqrt(2) * (op.conjugate().transpose() + op))
# Free Hamiltonian
H_k = 0
for a in a_b_list:
    H_k = H_k + a.conjugate().transpose() * a
# vacuum energy
H_k = H_k + 0.5 * Nmat * identity(L ** Nmat)

In [23]:
am = MatrixOp(a_b_list[0])

In [24]:
# this takes a really long time even for L=4
print(am.to_pauli_op())

SummedOp([
  0.5 * XIIIII,
  0.5j * YIIIII
])


The operation of using `PauliOp` directly to construct a single boson annihilation operator in the full space gives
```
0.5 * XIIIII
+ 0.5j * YIIIII
```


In [25]:
xm = MatrixOp(x_list[0])

In [26]:
print(xm.to_pauli_op())

0.7071067811865477 * XIIIII


The corresponding `PauliOp` is
```
0.7071067811865475 * XIIIII
```

### The result of computing the energy using the free Hamiltonian as a `MatrixOp`

In [33]:
Hm = MatrixOp(H_k)
result = npme.compute_minimum_eigenvalue(operator=Hm)

In [34]:
ref_value = result.eigenvalue.real
print(f'Ground state energy value: {ref_value:.5f}')

Ground state energy value: 3.00000


### Add interactions

In [68]:
v_b1 = ( x_list[2] * x_list[2] * x_list[3] * x_list[3]
    + x_list[2] * x_list[2] * x_list[4] * x_list[4]
    + x_list[1] * x_list[1] * x_list[3] * x_list[3]
    + x_list[1] * x_list[1] * x_list[5] * x_list[5]
    + x_list[0] * x_list[0] * x_list[4] * x_list[4]
    + x_list[0] * x_list[0] * x_list[5] * x_list[5]
    )
v_b2 = -2.0 * ( x_list[0] * x_list[2] * x_list[3] * x_list[5]
    + x_list[0] * x_list[1] * x_list[3] * x_list[4]
    + x_list[1] * x_list[2] * x_list[4] * x_list[5]
    )
V_b = v_b1 + v_b2


In [71]:
vm = MatrixOp(v_b1)

In [73]:
vm.to_pauli_op()

PauliOp(Pauli('IIIIII'), coeff=1.4999999999999998)

In [74]:
vm = MatrixOp(v_b2)

In [75]:
vm.to_pauli_op()

SummedOp([PauliOp(Pauli('IXXIXX'), coeff=-0.49999999999999994), PauliOp(Pauli('XIXXIX'), coeff=-0.49999999999999994), PauliOp(Pauli('XXIXXI'), coeff=-0.49999999999999994)], coeff=1.0, abelian=False)

In [69]:
# full hamiltonian
g2N = 0.2
H_full = H_k + g2N / N * V_b

In [77]:
Hm = MatrixOp(H_full)
Hm.to_pauli_op()

SummedOp([PauliOp(Pauli('IIIIII'), coeff=6.150000000000001), PauliOp(Pauli('ZIIIII'), coeff=-0.5), PauliOp(Pauli('IZIIII'), coeff=-0.5), PauliOp(Pauli('IIZIII'), coeff=-0.5), PauliOp(Pauli('IIIZII'), coeff=-0.5), PauliOp(Pauli('IIIIZI'), coeff=-0.5000000000000009), PauliOp(Pauli('IIIIIZ'), coeff=-0.5000000000000004), PauliOp(Pauli('XXIXXI'), coeff=-0.05), PauliOp(Pauli('XIXXIX'), coeff=-0.05), PauliOp(Pauli('IXXIXX'), coeff=-0.05)], coeff=1.0, abelian=False)

Using `PauliOp` with L=2 I get
```
SummedOp([PauliOp(Pauli('IIIIII'), coeff=6.150000000000002), PauliOp(Pauli('ZIIIII'), coeff=-0.5), PauliOp(Pauli('IZIIII'), coeff=-0.5), PauliOp(Pauli('IIZIII'), coeff=-0.5), PauliOp(Pauli('IIIZII'), coeff=-0.5), PauliOp(Pauli('IIIIZI'), coeff=-0.5), PauliOp(Pauli('IIIIIZ'), coeff=-0.5), PauliOp(Pauli('XXIXXI'), coeff=-0.04999999999999999), PauliOp(Pauli('XIXXIX'), coeff=-0.04999999999999999), PauliOp(Pauli('IXXIXX'), coeff=-0.04999999999999999)], coeff=1.0, abelian=False)
```

In [87]:
result = npme.compute_minimum_eigenvalue(operator=Hm)
ref_value = result.eigenvalue.real
print(f'Ground state energy value: {ref_value:.5f}')

Ground state energy value: 3.14808


In [88]:
print(result)

{   'aux_operator_eigenvalues': None,
    'eigenstate': VectorStateFn(Statevector([ 9.91404418e-01-1.28937624e-01j,
              1.19900511e-16-4.66415354e-17j,
              5.03621967e-18+3.89259549e-16j,
              3.40089855e-18-3.36715519e-17j,
             -3.46062058e-16-5.36833573e-17j,
             -5.33124977e-18+1.53240308e-17j,
             -5.07086228e-17+1.97815610e-17j,
              7.21486776e-17+7.44039916e-18j,
             -8.85964036e-17-4.40803312e-16j,
             -1.57752571e-17+4.15697822e-17j,
             -1.00461964e-17+4.03291685e-17j,
             -2.35697774e-18+1.62240065e-17j,
              5.79376070e-18+1.32449984e-17j,
             -1.26922057e-17+4.62396953e-18j,
              4.34680686e-18-3.93759151e-17j,
              7.46703160e-17+5.30164773e-17j,
              1.16780230e-16+1.90138948e-16j,
             -4.00234677e-17+1.07369239e-16j,
             -3.04308641e-18+6.95400088e-18j,
             -6.19854328e-17-1.44717768e-17j,
          